In [1]:
using Distributions, TimeIt, PyPlot

In [2]:
include("zz_structures.jl")
include("mbsampler.jl")

get_clustering_vec_and_score (generic function with 1 method)

In [12]:
d, Nobs = 10, 100
X = rand(Normal(), d, Nobs)
X[1,:] = ones(Nobs)
ξ_true = rand(Normal(),d)
y = [rand(Binomial(1, 1/(1+exp(-ξ_true'X[:,j]))), 1)[1] + 0. for j in 1:Nobs];

In [13]:
my_ll = ll_logistic(X,y);
my_prior = gaussian_prior_nh(zeros(d),1*ones(d))
my_model = model(my_ll, my_prior);

In [14]:
ϵ = 1e-5
weights_cv = zeros(d, Nobs)
for n in 1:Nobs
    weights_cv[:,n] = [abs.(X[i,n])*norm(X[:,n]) for i in 1:d] + ϵ
end
weights_cv ./= sum(weights_cv,2);

weights = abs.(X) + ϵ
weights ./= sum(weights,2);

In [15]:
A = eye(d)
mb_size = 10
opf = projopf(A, 100) 
opt = maxa_opt(10^5)
outp = outputscheduler(opf,opt) ;
#gs = umbsampler(Nobs, mb_size)

In [16]:
# Stratified sub-sampling (and weights)
ξ_0 = rand(Normal(),d)
root = find_root(my_model, ξ_0)
ll_pd_root_list = [partial_derivative_vec(my_model.ll, root, i, 1:Nobs) for i in 1:d]

cmbsamplers = Array{cmbsampler}(d)
N_cluster = mb_size
mode = "wc"
#weights = 

for dim in 1:d
    csamplers = Array{mbsampler}(N_cluster)
    clusters = get_clustering_vec(ll_pd_root_list[dim], N_cluster, mode)
    #print(clusters)
    scp = ones(N_cluster)
    for (ci, c) in enumerate(clusters)
        #csamplers[ci] = umbsampler(size(c)[1], scp[ci])
        csamplers[ci] = wumbsampler(size(c)[1], scp[ci], weights[dim,c])
    end
    #cmbsamplers[dim] = cmbsampler(csamplers, clusters, ones(Nobs)/Nobs)
    cmbsamplers[dim] = cmbsampler(csamplers, clusters, weights[dim,:])
end
gs_list = mbsampler_list(d,cmbsamplers);

In [17]:
start = time()
ZZ_sample(my_model, outp, gs_list)
print( round((time()-start)/60, 2), " minutes to run \n")

0.15 minutes to run 


In [18]:
n_samples = 10^4
@timeit samples = extract_samples(outp.opf.xi_skeleton, outp.opf.bt_skeleton, outp.opf.bt_skeleton[end]/n_samples);

100 loops, best of 3: 1.48 ms per loop


In [19]:
[compute_configT(my_model, samples, k) for k in 1:d]

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


10-element Array{Float64,1}:
 0.965922
 1.36186 
 0.812463
 0.978456
 0.6378  
 0.572526
 1.34262 
 0.86381 
 1.54849 
 1.18986 

In [20]:
print(size(outp.opf.bt_skeleton), " ")
print(outp.opf.bt_skeleton[end])

(1, 2904) 121.32211638893153